In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-wisconsin-data/data.csv


## Reading Dataset

In [46]:
df=pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')

## Details of dataset

In [47]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [48]:
df.shape

(569, 33)

In [49]:
df.dropna()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32


In [66]:
df.dtypes

id                           int64
diagnosis                    int64
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst     

In [50]:
df. describe

<bound method NDFrame.describe of            id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0      842302         M        17.99         10.38          122.80     1001.0   
1      842517         M        20.57         17.77          132.90     1326.0   
2    84300903         M        19.69         21.25          130.00     1203.0   
3    84348301         M        11.42         20.38           77.58      386.1   
4    84358402         M        20.29         14.34          135.10     1297.0   
..        ...       ...          ...           ...             ...        ...   
564    926424         M        21.56         22.39          142.00     1479.0   
565    926682         M        20.13         28.25          131.20     1261.0   
566    926954         M        16.60         28.08          108.30      858.1   
567    927241         M        20.60         29.33          140.10     1265.0   
568     92751         B         7.76         24.54           47.92      181

In [51]:
df['diagnosis'].values

array(['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'M', 'M', 'B', 'B', 'B',
       'M', 'B', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M

### Mapping the values of malignant and benign

In [52]:
df['diagnosis'] = df['diagnosis'].map({'M':0,'B': 1})

In [53]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [54]:
df.duplicated().sum()

0

In [55]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

## Splitting data for training and testing

In [56]:
x = df[['radius_mean' , 'texture_mean' , 'perimeter_mean' , 'area_mean']]
y = df['diagnosis']

In [57]:
x

,radius_mean,texture_mean,perimeter_mean,area_mean
0,17.99,10.38,122.80,1001.0
1,20.57,17.77,132.90,1326.0
2,19.69,21.25,130.00,1203.0
3,11.42,20.38,77.58,386.1
4,20.29,14.34,135.10,1297.0
...,...,...,...,...
564,21.56,22.39,142.00,1479.0
565,20.13,28.25,131.20,1261.0
566,16.60,28.08,108.30,858.1
567,20.60,29.33,140.10,1265.0


In [58]:
y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: diagnosis, Length: 569, dtype: int64

In [59]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4)

## Classification

In [60]:
clf = svm.SVC()

# Fit the classifier to the training data
clf.fit(x_train, y_train)

SVC()

In [61]:
pred=clf.predict(x_test)

### Checking for accuracy

In [65]:
y_test_flat = y_test.values.ravel() if hasattr(y_test, 'values') else y_test
pred_flat = pred.ravel() if hasattr(pred, 'ravel') else pred

# Calculate accuracy using accuracy_score
accuracy = accuracy_score(y_test_flat, pred_flat)

# Print the accuracy in percentage
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 87.72%


## Multi-Class Classification

In [67]:
clf = svm.SVC(decision_function_shape='ovo')

In [68]:
clf.fit(x_train,y_train)

SVC(decision_function_shape='ovo')

In [73]:
dec = clf.decision_function(x_train)

In [74]:
print("Decision Function Values for All Features:", dec)

Decision Function Values for All Features: [ 7.97930947e-01  4.90695611e-01  8.27849271e-01 -2.12404302e+00
  1.12593668e+00 -1.07653086e+00 -1.90944473e+00  1.22044504e+00
 -1.02468939e+00  1.14013829e+00  1.32635920e+00  1.30593100e+00
  1.11503626e+00 -9.97367022e-04  1.32201775e+00  1.07874603e+00
 -1.00034065e+00  1.10323287e+00 -1.36999879e+00  1.32401902e+00
  9.99978709e-01 -2.07599182e+00  7.93076033e-02  1.28298478e+00
  6.13243325e-01  1.27186558e+00 -1.23080455e+00 -1.85724820e+00
 -1.49963819e+00  1.26884221e+00  1.29449219e+00 -1.57675468e+00
  1.28789677e+00  1.32577564e+00  1.29704451e+00 -2.04644272e+00
 -1.95889791e+00  6.85159790e-03 -1.19753408e+00  1.32239601e+00
  8.06493363e-01  1.30810933e+00  1.20322690e+00  1.25635624e+00
  1.31629420e+00  1.07394904e+00 -2.02883877e+00  1.32676085e+00
  8.64153281e-01  1.19397852e+00 -2.11347831e+00  1.32218764e+00
  6.75923296e-01 -2.01721003e+00  1.28029952e+00  1.32088919e+00
  8.12857467e-01  1.21994496e+00  9.99695666e-0

## Linear SVC

In [78]:
linear_clf = svm.LinearSVC(dual="auto")

In [79]:
linear_clf = clf.fit(x_train,y_train)

In [80]:
dec = linear_clf.decision_function(x_train)

In [82]:
dec

array([ 7.97930947e-01,  4.90695611e-01,  8.27849271e-01, -2.12404302e+00,
        1.12593668e+00, -1.07653086e+00, -1.90944473e+00,  1.22044504e+00,
       -1.02468939e+00,  1.14013829e+00,  1.32635920e+00,  1.30593100e+00,
        1.11503626e+00, -9.97367022e-04,  1.32201775e+00,  1.07874603e+00,
       -1.00034065e+00,  1.10323287e+00, -1.36999879e+00,  1.32401902e+00,
        9.99978709e-01, -2.07599182e+00,  7.93076033e-02,  1.28298478e+00,
        6.13243325e-01,  1.27186558e+00, -1.23080455e+00, -1.85724820e+00,
       -1.49963819e+00,  1.26884221e+00,  1.29449219e+00, -1.57675468e+00,
        1.28789677e+00,  1.32577564e+00,  1.29704451e+00, -2.04644272e+00,
       -1.95889791e+00,  6.85159790e-03, -1.19753408e+00,  1.32239601e+00,
        8.06493363e-01,  1.30810933e+00,  1.20322690e+00,  1.25635624e+00,
        1.31629420e+00,  1.07394904e+00, -2.02883877e+00,  1.32676085e+00,
        8.64153281e-01,  1.19397852e+00, -2.11347831e+00,  1.32218764e+00,
        6.75923296e-01, -